In [1]:
# !unzip -uq "testset_pn.zip" -d "testset_pn/"

In [1]:
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import glob

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
path_positve='testset_pn/positive' # 기쁨
path_nopositive='testset_pn/non-positive' # 기쁨 아님

path_negative='testset_pn/negative' # 찡그림
path_nonegative='testset_pn/non-negative' # 찡그림 아님


la=len(os.listdir(path_positve))
lb=len(os.listdir(path_nopositive))

lc=len(os.listdir(path_negative))
ld=len(os.listdir(path_nonegative))
# lc=len(os.listdir(path_unkno)) 

print('positive에 저장된 파일의 개수:'+str(la))
print('no positive에 저장된 파일의 개수:'+str(lb))
print('negative에 저장된 파일의 개수:'+str(lc))
print('no negative에 저장된 파일의 개수:'+str(ld))

positive에 저장된 파일의 개수:5861
no positive에 저장된 파일의 개수:5800
negative에 저장된 파일의 개수:5857
no negative에 저장된 파일의 개수:5800


In [3]:
xsize=100
ysize=100

In [4]:
x=[]
y=[]

from keras.preprocessing import image
import numpy as np

def read_dir(path, label):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            
            y.append(label)
            x.append(img_tr)
        except:
            pass

In [5]:
# 기쁨 판정 모델
# read_dir(path_positve, 1) 
# read_dir(path_nopositive, 0) 

# 찡그림 판정 모델
read_dir(path_negative, 1) 
read_dir(path_nonegative, 0) 

In [6]:
x=np.array(x)
y=np.array(y)

In [7]:
print(x.shape)

(11657, 100, 100, 3)


In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)

input_shape = (xsize,ysize, 3)

# batch_size = 32
# num_classes = 2
# epochs = 15

In [9]:
print(x_test.shape)
print(y_test.shape)

(2332, 100, 100, 3)
(2332,)


In [10]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
# 공통 모델
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.applications.mobilenet_v2 import MobileNetV2  
from keras.applications.mobilenet import MobileNet

input_shape = (100, 100, 3)
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
# base_model.trainable=False
x = Flatten()(base_model.output)
dropout = Dropout(0.6)(x)
output = Dense(2, activation='softmax', name='predictions')(dropout)
model2 = Model(inputs=base_model.input, output=output)

model2.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model2.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [ ]:
history = model2.fit(x_train, y_train, validation_split=0.5, 
          epochs=30, batch_size=256, verbose=1)

acc = model2.evaluate(x_test, y_test)
print(acc)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('loss, acc')
plt.xlabel('epoch')
plt.ylabel('loss, acc')
plt.legend(['train loss', 'test loss', 'train acc', 'test acc'])
plt.show()

In [ ]:
# model.save('save/MobileNetV2(positive).h5')
# print('모델 저장이 완료되었습니다')

In [ ]:
# !unzip -uq "roi_1.zip" -d "dataset_pred/"

In [ ]:
test_path = "dataset_pred/E01" # 무표정
test_path2 = "dataset_pred/E02" # 기쁨 
test_path3 = "dataset_pred/E03" # 찡그림

X=[]

def read_dir1(path):
    
    files = glob.glob(path + "/*.jpg")
    if bool(files) == False:
        files = glob.glob(path + "/*.png") 
     
    for f in files:

        try:
            img = image.load_img(f, target_size=(xsize,ysize))
            img_tr= image.img_to_array(img)        
            img_tr /= 255.
            X.append(img_tr)
        except:
            pass

In [ ]:
# test set(무표정)
read_dir1(test_path)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 무표정)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

In [ ]:
# test set(기쁨)
read_dir1(test_path2)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 기쁨)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 0:
        co+=1

print('\n', "%.2f" % (co/2880.))

In [ ]:
# test set(찡그림)
read_dir1(test_path3)
X = np.array(X)
pred = model.predict(X)

# 예측(test set 찡그림)
co=0
for i in range(2880):
    print(np.argmax(pred[i]),end='')
    tmp=np.argmax(pred[i])
    
    if tmp == 1:
        co+=1

print('\n', "%.2f" % (co/2880.))

In [ ]:
# E02 데이터셋 정제 후 

# acc : 무표정(0 %), 기쁨(58 %), 찡그림(41 %)